- **Importing the required libraries and set up the MySQL connection:**

In [1]:
# Imports:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse as up
import json

In [2]:
# MySQL Credentials
username = 'root'
password = up.quote_plus('OscarRamirez123@')
host = 'localhost'
database = 'movies'

In [3]:
# Create the connection string
connection_str = f'mysql+pymysql://{username}:{password}@{host}/{database}'

In [4]:
# Create the engine
engine = create_engine(connection_str)

- **Loading the data files and perform necessary transformations:**

In [5]:
# Load the title_basics data into a DataFrame
df_title_basics = pd.read_csv("title_basics-Copy1.csv")

In [6]:
# Normalize the genres column
df_title_basics['genres'] = df_title_basics['genres'].str.split(',')

In [7]:
# Step 2.1: Create the title_genres DataFrame
df_title_genres = df_title_basics.explode('genres')[['tconst', 'genres']]

In [8]:
# Step 2.2: Create the genres table
df_genres = pd.DataFrame(df_title_genres['genres'].unique(), columns=['genre_name'])
df_genres['genre_id'] = df_genres.index + 1

"""Note: I added 1 to the index while creating the 'genre_id' column because in Python, 
          the index starts from 0 by default. Adding 1 to the index ensures that the genre_id values start from 1, 
          as expected for primary keys in database tables."""

"Note: I added 1 to the index while creating the 'genre_id' column because in Python, \n          the index starts from 0 by default. Adding 1 to the index ensures that the genre_id values start from 1, \n          as expected for primary keys in database tables."

In [9]:
# Step 2.3: Merge the title_genres DataFrame with the genres table
df_title_genres = df_title_genres.merge(df_genres, left_on='genres', right_on='genre_name')[['tconst', 'genre_id']]

- **Insert data into the "genres" and "title_genres" tables:**

In [10]:
# Insert data into the genres table
df_genres.to_sql('genres', engine, if_exists='replace', index=False)

25

In [11]:
# Insert data into the title_genres table
df_title_genres.to_sql('title_genres', engine, if_exists='replace', index=False)

153249

In [12]:
# Verify the tables
print(pd.read_sql_query('SELECT * FROM genres LIMIT 5', engine))
print(pd.read_sql_query('SELECT * FROM title_genres LIMIT 5', engine))

  genre_name  genre_id
0     Comedy         1
1    Fantasy         2
2    Romance         3
3      Drama         4
4    History         5
      tconst  genre_id
0  tt0035423         1
1  tt0088751         1
2  tt0100275         1
3  tt0108549         1
4  tt0118652         1


- **Read the remaining data files and load them to the created "tmdb_data" table:**

In [13]:
# Load the title_ratings data into a DataFrame
df_title_ratings = pd.read_csv("title_ratings.csv")

In [14]:
folder = "Data/"

with open(f"{folder}tmdb_api_results_2000.json", "r") as f:
    tmdb_api_results_2000 = json.load(f)

with open(f"{folder}tmdb_api_results_2001.json", 'r') as a:
    tmdb_api_results_2001 = json.load(a)

In [15]:
# Create the tmdb_data DataFrame for the years 2000 and 2001
df_tmdb_data = pd.DataFrame(tmdb_api_results_2000 + tmdb_api_results_2001)

In [16]:
# Select only the required columns for tmdb_data table
df_tmdb_data = df_tmdb_data[['imdb_id', 'revenue', 'budget', 'certification']]

In [17]:
# Step 4.1: Create the tmdb_data table
df_tmdb_data.to_sql('tmdb_data', engine, if_exists='replace', index=False)

2579

In [18]:
# Verify the first 5 rows of the tmdb_data table
print(pd.read_sql_query('SELECT * FROM tmdb_data LIMIT 5', engine))

     imdb_id     revenue      budget certification
0          0         NaN         NaN          None
1  tt0113026         0.0  10000000.0              
2  tt0113092         0.0         0.0              
3  tt0116391         0.0         0.0          None
4  tt0118694  14204632.0    150000.0            PG


- **Create the "title_basics" and "title_ratings" tables:**

In [19]:
# Convert the "genres" column from list to comma-separated string
df_title_basics['genres'] = df_title_basics['genres'].apply(lambda x: ','.join(x))

In [20]:
from sqlalchemy import VARCHAR, TEXT, FLOAT, INTEGER

# Create the title_basics table with the updated "genres" column
df_title_basics.to_sql('title_basics', engine, dtype={
    'tconst': VARCHAR(20),
    'titleType': VARCHAR(20),
    'primaryTitle': TEXT(),
    'originalTitle': TEXT(),
    'isAdult': INTEGER(),
    'startYear': FLOAT(),
    'endYear': FLOAT(),
    'runtimeMinutes': INTEGER(),
    'genres': TEXT() 
}, if_exists='replace', index=False)
print("Done after a 1,000 errors")

Done after a 1,000 errors


In [21]:

"""Helpful Link:""" 
#                https://docs.sqlalchemy.org/en/20/core/type_basics.html
#                https://docs.sqlalchemy.org/en/20/core/exceptions.html
"""ChatGPT examples:""" #https://chat.openai.com/share/9d6b75d1-d53c-4a69-9df8-43c608e0ba29


'ChatGPT examples:'

In [22]:
# Step 5.2: Set "tconst" as the primary key for title_basics table
engine.execute('ALTER TABLE title_basics ADD PRIMARY KEY (`tconst`);')

In [23]:
# Verify the first 5 rows of the title_basics table
print(pd.read_sql_query('SELECT * FROM title_basics LIMIT 5', engine))

      tconst titleType                                       primaryTitle  \
0  tt0035423     movie                                     Kate & Leopold   
1  tt0043139     movie                        Life of a Beijing Policeman   
2  tt0062336     movie  The Tango of the Widower and Its Distorting Mi...   
3  tt0069049     movie                         The Other Side of the Wind   
4  tt0088751     movie                                  The Naked Monster   

                               originalTitle  isAdult  startYear endYear  \
0                             Kate & Leopold        0     2001.0    None   
1                           Wo zhe yi bei zi        0     2013.0    None   
2  El tango del viudo y su espejo deformante        0     2020.0    None   
3                 The Other Side of the Wind        0     2018.0    None   
4                          The Naked Monster        0     2005.0    None   

   runtimeMinutes                  genres  
0             118  Comedy,Fantasy,Ro

In [25]:
# Convert the "tconst" column to string data type
df_title_ratings['tconst'] = df_title_ratings['tconst'].astype(str)

In [26]:
# Create the title_ratings table with the updated "tconst" column
df_title_ratings.to_sql('title_ratings', engine, dtype={
    'tconst': VARCHAR(20),
    'averageRating': FLOAT(),
    'numVotes': INTEGER()
}, if_exists='replace', index=False)
print("Another one Done")

Another one Done


In [27]:
# Step 5.4: Set "tconst" as the primary key for title_ratings table
engine.execute('ALTER TABLE title_ratings ADD PRIMARY KEY (`tconst`);')

In [28]:
# Verify the first 5 rows of the title_ratings table
print(pd.read_sql_query('SELECT * FROM title_ratings LIMIT 5', engine))

      tconst  averageRating  numVotes
0  tt0000001            5.7      1982
1  tt0000002            5.8       265
2  tt0000005            6.2      2624
3  tt0000006            5.1       182
4  tt0000007            5.4       821


- **Show all the tables in the database:**

In [29]:
# Show all tables in the database
print(pd.read_sql_query('SHOW TABLES', engine))

  Tables_in_movies
0           genres
1     title_basics
2     title_genres
3    title_ratings
4        tmdb_data


In [30]:
print("Done, and this time I read the instructions from head to toe ; )")

Done, and this time I read the instructions from head to toe ; )
